# Function Call

In [12]:
import json
def add(x, y):
    return json.dumps({"result" : x + y})

def subtract(x, y):
    return json.dumps({"result" : x - y})

def multiply(x, y):
    return json.dumps({"result" : x * y})

def divide(x, y):
    if y == 0:
        return "Error! Division by zero."
    return json.dumps({"result" : x / y})

def get_tools_definition(function_name, description):
    return {
            "type": "function",
            "function": {
                "name": function_name,
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The first number",
                        },
                        "y": {
                            "type": "number",
                            "description": "the second number",
                        },
                    },
                    "required": ["x", "y"],
                },
            },
        }

tools = [
    get_tools_definition("add", "add x to y"),
    get_tools_definition("subtract", "subtract y from x"),
    get_tools_definition("multiply", "x multiply y"),
    get_tools_definition("divide", "x divide y"),
]


In [13]:

from openai import OpenAI
import json

client = OpenAI()


messages = [{"role": "user", "content": "What does 1024 + 10086 equal to?"}]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)


In [14]:
response_message = response.choices[0].message
print(response_message.content)

None


In [15]:
tool_calls = response_message.tool_calls
available_functions = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide,
}

In [16]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    tool_call_id = tool_call.id
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        x=function_args.get("x"),
        y=function_args.get("y"),
    )
    print(function_response)

{"result": 11110}


In [17]:
messages.append(response_message)
messages.append(
    {
        "tool_call_id": tool_call_id,
        "role": "tool",
        "name": function_name,
        "content": str(function_response),
    }
)  

final_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
)
print(final_response.choices[0].message.content)

1024 + 10086 = 11110


## Incorrect Function Description

In [18]:
def chat_using_function_call(content, tools = tools, available_functions = available_functions, client = client):
    messages = [{"role": "user", "content": content}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    messages.append(response_message)    
    tool_calls = response_message.tool_calls
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            x=function_args.get("x"),
            y=function_args.get("y"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
            )
    final_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
    )
    print(final_response.choices[0].message.content)

In [19]:
chat_using_function_call("What does 10086 - 1024 equal to?")

10086 - 1024 equals 9062.


In [20]:
tools = [
    get_tools_definition("add", "add x to y"),
    get_tools_definition("subtract", "y - x"),
    get_tools_definition("multiply", "x multiply y"),
    get_tools_definition("divide", "x divide y"),
]


In [21]:
chat_using_function_call("What does 10086 - 1024 equal to?", tools = tools)

10086 - 1024 equals 9062.


## SerpAPI

In [22]:
API_KEY = "YOUR_API_KEY"

from serpapi.google_search import GoogleSearch

params = {
  "engine": "google_shopping",
  "q": "Macbook M3",
  "api_key": API_KEY
}
search = GoogleSearch(params)
results = search.get_dict()
shopping_results = results["shopping_results"]


In [23]:
import json
pretty_json = json.dumps(shopping_results, indent=4)
print(pretty_json)

[
    {
        "position": 1,
        "title": "Apple 14\" MacBook Pro (M3, Space Gray) with Apple M3 8-Core Chip 8GB Unified RAM ...",
        "link": "https://www.bhphotovideo.com/c/product/1793630-REG/apple_mtl73ll_a_14_macbook_pro_with.html?kw=APMTL73LLA&ap=y&smp=y&BI=E6540&srsltid=AfmBOopxJGtKB93QRtNQIcVcZ6ExpmkCUFj_tri2hicX0iQ02Jwvegnii78",
        "product_link": "https://www.google.com/shopping/product/1?gl=us&prds=pid:10805795665197980642",
        "product_id": "10805795665197980642",
        "serpapi_product_api": "https://serpapi.com/search.json?device=desktop&engine=google_product&gl=us&google_domain=google.com&hl=en&product_id=10805795665197980642",
        "source": "B&H Photo-Video-Audio",
        "price": "$1,449.00",
        "extracted_price": 1449.0,
        "old_price": "$1,599.00",
        "extracted_old_price": 1599.0,
        "rating": 5.0,
        "reviews": 2,
        "extensions": [
            "Mac OS",
            "Octa Core",
            "USB-C",
         

In [31]:
def search_product(product_keywords):

    params = {
    "engine": "google_shopping",
    "q": product_keywords,
    "api_key": API_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    shopping_results = results["shopping_results"][0]
    return json.dumps(shopping_results)

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_product",
            "description": "search for a product on google shopping, get information like name, price, description etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_keywords": {
                        "type": "string",
                        "description": "Name or key words of the product to search",
                    },
                },
                "required": ["product_keywords"],
            },
        }
    },
]

In [38]:
available_functions = {
    "search_product": search_product,
}

def chat_using_function_call(content, tools = tools, available_functions = available_functions, client = client):
    messages = [{"role": "user", "content": content}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    messages.append(response_message)    
    tool_calls = response_message.tool_calls
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            product_keywords=function_args.get("product_keywords"),
        )
        print("function " + function_name + ", with arguments " + str(function_args) + " called")
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
            )
    final_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
    )
    print(final_response.choices[0].message.content)

In [39]:
chat_using_function_call("What is the price of Macbook M3?", tools = tools)

function search_product, with arguments {'product_keywords': 'Macbook M3'} called
The price of the Apple 14" MacBook Pro with M3 8-Core Chip and 8GB Unified RAM is $1,449.00. You can find more details on the product [here](https://www.bhphotovideo.com/c/product/1793630-REG/apple_mtl73ll_a_14_macbook_pro_with.html?kw=APMTL73LLA&ap=y&smp=y&BI=E6540&srsltid=AfmBOopxJGtKB93QRtNQIcVcZ6ExpmkCUFj_tri2hicX0iQ02Jwvegnii78).


In [40]:
chat_using_function_call("Could you help me to compare the price of Macbook M3 and Macbook M2?", tools = tools)

function search_product, with arguments {'product_keywords': 'Macbook M3'} called
function search_product, with arguments {'product_keywords': 'Macbook M2'} called
The price of the Macbook M3 (Apple 14" MacBook Pro with M3 8-Core Chip 8GB Unified RAM) is $1,449.00. You can find more details about it [here](https://www.bhphotovideo.com/c/product/1793630-REG/apple_mtl73ll_a_14_macbook_pro_with.html?kw=APMTL73LLA&ap=y&smp=y&BI=E6540&srsltid=AfmBOopxJGtKB93QRtNQIcVcZ6ExpmkCUFj_tri2hicX0iQ02Jwvegnii78).

The price of the Macbook M2 (MacBook Air 15-Inch, M2 8GB RAM 256GB SSD) is $999.00. More details can be found [here](https://www.bestbuy.com/site/apple-macbook-air-15-laptop-m2-chip-8gb-memory-256gb-ssd-midnight/6534606.p?skuId=6534606&utm_source=feed).

Therefore, the Macbook M3 is priced higher than the Macbook M2.
